In [1]:
# discharge.py usage
# by Rui, Aug 2021

In [1]:
import pickle
from offline import discharge
from offline import ReadObs
from offline import ReadOb
from offline import ReadPRD
from offline import ReadQparams
"""
inputs:
    ----------
    reach:
        - reach['area_fits'] contains the parameters of cross-sectional area change  
        - reach['discharge_models'] contains the parameters of 5 discharge models  
        
    reach_stats: 
        - reach_stats['height'], SWOT reach height [m] 
        - reach_stats['width'], SWOT reach width [m]
        - reach_stats['slope'], SWOT reach slope [m/m]
        
outputs:
    ----------
    d_x_area: cross-sectional area change [m^2]
    d_x_area_u: cross-sectional area change uncertainty [m^2]
    metro_q_c : MetroMan discharge gage constrained [m^3/s]
    metro_q_uc : MetroMan discharge gage un-constrained [m^3/s]
    bam_q_c : BAM discharge gage constrained [m^3/s]
    bam_q_uc : BAM discharge gage un-constrained [m^3/s]
    bam_q_c : BAM discharge gage constrained [m^3/s]
    bam_q_uc : BAM discharge gage un-constrained [m^3/s]
    hivdi_q_c : HiVDI discharge gage constrained [m^3/s]
    hivdi_q_uc : HiVDI discharge gage un-constrained [m^3/s]
    momma_q_c : MOMMA discharge gage constrained [m^3/s]
    momma_q_uc : MOMMA discharge gage un-constrained [m^3/s]
    sads_q_c : SADS discharge gage constrained [m^3/s]
    sads_q_uc : SADS discharge gage un-constrained [m^3/s]
    
    **Discharge uncertainties are not implemented at this moment
"""

# load data saved from the RiverObs run on Sac reaches
# note: some of the items under the dict provided in the /data won't be used
obs = 'C:/Users/wei.263/Desktop/Github/offline-discharge-data-product-creation/data/sac_pkl_files/reach25.pkl'
prd ='C:/Users/wei.263/Desktop/Github/offline-discharge-data-product-creation/data/sac_pkl_files/reach_prd25.pkl'
reach_stats = pickle.load(open(obs, "rb"))
reach = pickle.load(open(prd, "rb"))

In [2]:
# # example 1. run discharge module using the .pkl data saved from RiverObs runs
# # copied from RiverObs/src/SWOTRiver
# discharge_model_values = discharge.compute(
#             reach, reach_stats['height'], reach_stats['width'],
#             reach_stats['slope'])


# # print(discharge_model_values)

In [3]:
# # example 2. run discharge module using getting the data saved from SWORD/PRD and rivertile
# # read swot rivertile
# rivertile_path = '/Users/rui/Desktop/Github/offline-discharge-data-product-\
# creation/data/rivertile/rivertile.nc'
# obs = ReadObs.Rivertile(rivertile_path)

# # read prd/sword
# sword_path = '/Users/rui/Desktop/Github/offline-discharge-data-product-\
# creation/data/sword/na_apriori_rivers_v07_4q_v8.nc'
# for i in range(len(obs['reach_id'])):
#     prd = ReadPRD.ReachDatabase(sword_path, obs['reach_id'][i])

#     if prd['area_fits']['h_w_nobs'] != -9999:
#         discharge_model_values = discharge.compute(
#                 prd, obs['height'][i], obs['width'][i],
#                 obs['slope'][i])
#         #print(discharge_model_values)

In [4]:
# # example 3. run discharge module using getting the data saved from SWORD/PRD and single reach
# # read swot reach
# rivertile_dir = '/Users/rui/Desktop/Github/offline-discharge-data-product-creation/data/swot/'
# reach_id = 77449100061
# ob = ReadOb.Rivertile(rivertile_dir)

# # read prd/sword
# sword_path = '/Users/rui/Desktop/Github/offline-discharge-data-product-\
# creation/data/sword/na_apriori_rivers_v07_4q_v8.nc'

# prd = ReadPRD.ReachDatabase(sword_path, ob['reach_id'])
    
# for i in range(len(ob['height'])):
    
#     if prd['area_fits']['h_w_nobs'] != -9999:
#         discharge_model_values = discharge.compute(
#                 prd, ob['height'][i], ob['width'][i],
#                 ob['slope'][i])
#         print(i, prd,'obs',obs['height'][i])

In [1]:
# example 4. extract Q params from Confluence
# compute d_x_area and q uncertainties in discharge.py
from pathlib import Path
from offline import discharge
from offline import ReadOb
from offline import ReadQparams
from offline.ReadQparams import extract_alg
from offline import ReadPRD


r_id = "77449100061"
run_type = 'constrained'

# read obs
rivertile_dir = 'C:/Users/wei.263/Desktop/Github/offline-discharge-data-product-creation/data/swot_4/'
rivertile_file = Path(rivertile_dir + '/' + f'{r_id}_SWOT.nc')

ob = ReadOb.Rivertile(rivertile_file)
#print(ob)

# read prd/sword, only constrained q variables are populated 
sword_path = 'C:/Users/wei.263/Desktop/Github/offline-discharge-data-product-creation\
/data/sword/na_apriori_rivers_v07_4q_v8.nc'

priors = ReadPRD.ReachDatabase(sword_path, ob['reach_id'])
FLPE_DIR = "C:/Users/wei.263/Desktop/Github/offline-discharge-data-product-creation/data/flpe"
# if run_type:
#     priors["discharge_models"] = extract_alg(FLPE_DIR, ob["reach_id"], run_type)


In [2]:
# run discharge algs
for i in range(len(ob['height'])):
    
    if priors['discharge_models']['constrained']['BAM']['n'] is not None:
        discharge_model_values = discharge.compute(
                priors, ob['height'][i], ob['width'][i],
                ob['slope'][i], ob['d_x_area'][i],
                ob['wse_u'][i], ob['width_u'][i],
                ob['slope_u'][i], ob['d_x_area_u'][i])
        print('cycle', i, ':', 
              '\nmetroman', discharge_model_values['metro_q_c'], discharge_model_values['metro_q_c_u'],
              '\nbam', discharge_model_values['bam_q_c'], discharge_model_values['bam_q_c_u'],
              '\nhivdi', discharge_model_values['hivdi_q_c'], discharge_model_values['hivdi_q_c_u'],
              '\nmomma', discharge_model_values['momma_q_c'], discharge_model_values['momma_q_c_u'],
              '\nsads', discharge_model_values['sads_q_c'], discharge_model_values['sads_q_c_u'])


computing d_x_area
cycle 0 : 
metroman [116.17553092] [0.43444056] 
bam [119.35875707] [0.41368916] 
hivdi [116.17764586] [0.41549165] 
momma [113.57272553] [0.40907229] 
sads [119.35875707] [0.41368916]
computing d_x_area
cycle 1 : 
metroman -999999999999 -999999999999 
bam -999999999999 -999999999999 
hivdi -999999999999 -999999999999 
momma -999999999999 -999999999999 
sads -999999999999 -999999999999
computing d_x_area
cycle 2 : 
metroman [110.57817434] [0.43682777] 
bam [117.77579492] [0.41373948] 
hivdi [110.58146704] [0.41599392] 
momma [105.971098] [0.40886441] 
sads [117.77579492] [0.41373948]
computing d_x_area
cycle 3 : 
metroman [119.47018075] [0.42969123] 
bam [106.58255914] [0.41430534] 
hivdi [119.46792405] [0.41553776] 
momma [146.22189646] [0.41009834] 
sads [106.58255914] [0.41430534]
computing d_x_area
cycle 4 : 
metroman -999999999999 -999999999999 
bam -999999999999 -999999999999 
hivdi -999999999999 -999999999999 
momma -999999999999 -999999999999 
sads -999999999

C:\Users\wei.263\Desktop\Github\offline-discharge-data-product-creation\offline\discharge.py:271: RuntimeWarning: invalid value encountered in greater_equal
  observed_height >= height_fits_ll,
C:\Users\wei.263\Desktop\Github\offline-discharge-data-product-creation\offline\discharge.py:272: RuntimeWarning: invalid value encountered in less
  observed_height < height_fits_ul))
C:\Users\wei.263\Desktop\Github\offline-discharge-data-product-creation\offline\discharge.py:128: RuntimeWarning: invalid value encountered in less_equal
  if reach_height <= momma_H:
C:\Users\wei.263\Desktop\Github\offline-discharge-data-product-creation\offline\discharge.py:121: RuntimeWarning: invalid value encountered in greater
  if momma_Save > 0 and momma_H > momma_B and reach_height > momma_B:


 [0.41388501] 
hivdi [190.52126906] [0.41381578] 
momma [129.05344299] [0.41120926] 
sads [198.80255549] [0.41388501]
computing d_x_area
cycle 10 : 
metroman -999999999999 -999999999999 
bam -999999999999 -999999999999 
hivdi -999999999999 -999999999999 
momma -999999999999 -999999999999 
sads -999999999999 -999999999999
computing d_x_area
cycle 11 : 
metroman [193.35797512] [0.42553698] 
bam [196.63153262] [0.41319809] 
hivdi [193.35829572] [0.41311587] 
momma [138.07218782] [0.4105229] 
sads [196.63153262] [0.41319809]
computing d_x_area
cycle 12 : 
metroman [134.64246386] [0.4322951] 
bam [142.3806727] [0.41250477] 
hivdi [134.64520455] [0.41368127] 
momma [112.66276862] [0.40841005] 
sads [142.3806727] [0.41250477]
computing d_x_area
cycle 13 : 
metroman -999999999999 -999999999999 
bam -999999999999 -999999999999 
hivdi -999999999999 -999999999999 
momma -999999999999 -999999999999 
sads -999999999999 -999999999999
computing d_x_area
cycle 14 : 
metroman [186.52580203] [0.4237213]

In [4]:
#print(priors)